# Turnover-Analysis

In [2]:
import os
import pyspark
import pandas as pd
import pyspark.sql.functions as f
import plotly.express as px
import plotly.graph_objects as go

In [3]:
spark = pyspark.sql.SparkSession.builder.appName("app1").getOrCreate()
sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)

In [4]:
# Datatypes
sdf = sdf.withColumn("event_time", sdf["event_time"].cast(pyspark.sql.types.TimestampType()))
sdf = sdf.withColumn("category_id", sdf["category_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("product_id", sdf["product_id"].cast(pyspark.sql.types.StringType()))
sdf = sdf.withColumn("user_id", sdf["user_id"].cast(pyspark.sql.types.StringType()))

# Feature Splitting
sdf = sdf.withColumn("category_class", f.substring_index(sdf.category_code, '.', 1))

# sdf = sdf.withColumn("category_class", f.split(sdf["category_code"], ".").getItem(0))
# sdf = sdf.withColumn("category_sub_class", f.split(sdf["category_code"], ".").getItem(1))
# sdf = sdf.withColumn("category_sub_sub_class", f.split(sdf["category_code"], ".").getItem(2))

sdf = sdf.withColumn("year", f.year("event_time"))
sdf = sdf.withColumn("month", f.month("event_time"))
sdf = sdf.withColumn("weekofyear", f.weekofyear("event_time"))
sdf = sdf.withColumn("dayofyear", f.dayofyear("event_time"))
sdf = sdf.withColumn("dayofweek", f.dayofweek("event_time"))
sdf = sdf.withColumn("dayofmonth", f.dayofmonth("event_time"))

# None Handling
sdf = sdf.fillna(value="not defined")


In [5]:
sdf = sdf.withColumn('turnover', f.when(f.col('event_type') == 'purchase', f.col('price')).otherwise(0))
sdf = sdf.withColumn('bougth_quantity', f.when(f.col('event_type') == 'purchase', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('viewed_quantity', f.when(f.col('event_type') == 'view', f.lit(1)).otherwise(0))
sdf = sdf.withColumn('cart_quantity', f.when(f.col('event_type') == 'cart', f.lit(1)).otherwise(0))

In [6]:
sdf_raw = sdf
sdf.createOrReplaceTempView("Data")
sdf_raw.show()

+-------------------+----------+----------+-------------------+--------------------+-----------+------+---------+--------------------+--------------+----+-----+----------+---------+---------+----------+--------+---------------+---------------+-------------+
|         event_time|event_type|product_id|        category_id|       category_code|      brand| price|  user_id|        user_session|category_class|year|month|weekofyear|dayofyear|dayofweek|dayofmonth|turnover|bougth_quantity|viewed_quantity|cart_quantity|
+-------------------+----------+----------+-------------------+--------------------+-----------+------+---------+--------------------+--------------+----+-----+----------+---------+---------+----------+--------+---------------+---------------+-------------+
|2019-11-01 01:00:00|      view|   1003461|2053013555631882655|electronics.smart...|     xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|   electronics|2019|   11|        44|      305|        6|         1|     0.0|              0

## Produkte und Umsatz

In [7]:
 sdf_product_overview = spark.sql("SELECT Product_id, \
                                MEAN(price), \
                                SUM(viewed_quantity) ,\
                                SUM(cart_quantity),\
                                SUM(bougth_quantity), \
                                SUM(turnover) AS Turnover\
                        FROM Data \
                        GROUP BY Product_id \
                        ORDER BY SUM(turnover) DESC")
sdf_product_overview.show()

+----------+-----------+--------------------+------------------+--------------------+--------+
|Product_id|mean(price)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Turnover|
+----------+-----------+--------------------+------------------+--------------------+--------+
|  13200605|      566.3|                   2|                 0|                   1|   566.3|
|   1005161|     211.92|                   1|                 0|                   1|  211.92|
|   2300214|     386.08|                   1|                 0|                   0|     0.0|
|   4804194|      69.24|                   1|                 0|                   0|     0.0|
|  15100148|     378.36|                   1|                 0|                   0|     0.0|
|  25600085|     134.37|                   1|                 0|                   0|     0.0|
|   3701016|     108.09|                   1|                 0|                   0|     0.0|
|  12708306|      30.43|                   1|     

In [8]:
px.bar(sdf_product_overview.limit(10).toPandas(), x='Product_id', y='Turnover')

In [8]:
 sdf_brand_overview = spark.sql("SELECT brand, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY brand \
                                ORDER BY Turnover DESC")
sdf_brand_overview.show()


+-----------+-----------------+------------------+--------------------+------------------+--------------------+------+
|      brand|count(Product_id)|       mean(price)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Umsatz|
+-----------+-----------------+------------------+--------------------+------------------+--------------------+------+
|not defined|               41| 176.8419512195122|                  40|                 0|                   1| 566.3|
|     xiaomi|               15|224.28133333333332|                  14|                 0|                   1|211.92|
|      oasis|                1|             32.66|                   1|                 0|                   0|   0.0|
|    element|                1|            133.83|                   1|                 0|                   0|   0.0|
|      epson|                1|            203.84|                   1|                 0|                   0|   0.0|
|   goodloot|                1|              8.7

In [9]:
px.bar(sdf_brand_overview.limit(10).toPandas(), x='brand', y='Turnover')

In [10]:
 sdf_category_id_overview = spark.sql("SELECT category_id, \
                                                COUNT(Product_id), \
                                                MEAN(price), \
                                                SUM(viewed_quantity) ,\
                                                SUM(cart_quantity),\
                                                SUM(bougth_quantity), \
                                                SUM(turnover) AS Turnover\
                                        FROM Data \
                                        GROUP BY category_id \
                                        ORDER BY Turnover DESC")
sdf_category_id_overview.show()

+-------------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|        category_id|count(Product_id)|       mean(price)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Umsatz|
+-------------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|2053013557192163841|                3|             566.3|                   2|                 0|                   1| 566.3|
|2053013555631882655|               38| 474.5957894736841|                  36|                 1|                   1|211.92|
|2053013558190408249|                1|             30.86|                   1|                 0|                   0|   0.0|
|2134905044833666047|                2|135.60500000000002|                   2|                 0|                   0|   0.0|
|2053013558920217191|               18| 924.2788888888889|                  18|                 0|             

In [11]:
px.bar(sdf_category_id_overview.limit(10).toPandas(), x='category_id', y='Turnover')

In [12]:
 sdf_category_code_overview = spark.sql("SELECT category_code, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY category_code \
                                ORDER BY Turnover DESC")
sdf_category_code_overview.show()

+--------------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|       category_code|count(Product_id)|       mean(price)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Umsatz|
+--------------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|furniture.bedroom...|                4|           458.825|                   3|                 0|                   1| 566.3|
|electronics.smart...|               38| 474.5957894736841|                  36|                 1|                   1|211.92|
|construction.tool...|                4|          130.7775|                   4|                 0|                   0|   0.0|
|construction.tool...|                1|            254.78|                   1|                 0|                   0|   0.0|
|furniture.kitchen...|                6|39.248333333333335|                   6|                 0|     

In [13]:
px.bar(sdf_category_code_overview.limit(10).toPandas(), x='category_code', y='Turnover')

In [14]:
 sdf_category_class_overview = spark.sql("SELECT category_class, \
                                        COUNT(Product_id), \
                                        MEAN(price), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY category_class \
                                ORDER BY Turnover DESC")
sdf_category_class_overview.show()

+--------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|category_class|count(Product_id)|       mean(price)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Umsatz|
+--------------+-----------------+------------------+--------------------+------------------+--------------------+------+
|     furniture|               14|241.24142857142857|                  13|                 0|                   1| 566.3|
|   electronics|               57|446.76929824561387|                  55|                 1|                   1|211.92|
|          auto|                7|192.08428571428573|                   7|                 0|                   0|   0.0|
|    appliances|               14|341.70214285714286|                  14|                 0|                   0|   0.0|
|       apparel|                6|52.251666666666665|                   6|                 0|                   0|   0.0|
|   not defined|        

In [15]:
px.bar(sdf_category_class_overview.limit(10).toPandas(), x='category_class', y='Turnover')

## Users und Umsatz

In [16]:
 sdf_user_overview = spark.sql("SELECT user_id, \
                                        COUNT(user_session), \
                                        SUM(viewed_quantity) ,\
                                        SUM(cart_quantity),\
                                        SUM(bougth_quantity), \
                                        SUM(turnover) AS Turnover\
                                FROM Data \
                                GROUP BY user_id \
                                ORDER BY Turnover DESC")
sdf_user_overview.show()

+---------+-------------------+--------------------+------------------+--------------------+------+
|  user_id|count(user_session)|sum(viewed_quantity)|sum(cart_quantity)|sum(bougth_quantity)|Umsatz|
+---------+-------------------+--------------------+------------------+--------------------+------+
|559368633|                  3|                   2|                 0|                   1| 566.3|
|513351129|                  2|                   1|                 0|                   1|211.92|
|519041881|                  1|                   1|                 0|                   0|   0.0|
|512416379|                  2|                   2|                 0|                   0|   0.0|
|566280399|                  1|                   1|                 0|                   0|   0.0|
|515240495|                  1|                   1|                 0|                   0|   0.0|
|518045858|                  2|                   2|                 0|                   0|   0.0|


In [17]:
px.bar(sdf_user_overview.limit(10).toPandas(), x='user_id', y='Turnover')

## Zeit und Umsatz

In [18]:

sdf_month_Umsatz = spark.sql("SELECT month, \
                                    dayofmonth, \
                                    SUM(turnover) AS Turnover\
                            FROM Data \
                            GROUP BY month, dayofmonth \
                            ORDER BY Turnover DESC")
sdf_month_Umsatz.show()


+-----+----------+-----------------+
|month|dayofmonth|           Umsatz|
+-----+----------+-----------------+
|   11|         1|778.2199999999999|
+-----+----------+-----------------+



In [19]:
df = sdf_month_Umsatz.toPandas()
fig = go.Figure()
fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Day of Month"),
    yaxis=dict(title_text="Count"),
    barmode="stack",
)

fig.add_trace(
        go.Bar(x=[df.dayofmonth, df.month], y=df.Turnover),
    )
fig

In [20]:
sdf_week_Umsatz = spark.sql("SELECT weekofyear, \
                                    dayofweek, \
                                    SUM(turnover) AS Turnover\
                            FROM Data \
                            GROUP BY weekofyear, dayofweek \
                            ORDER BY Turnover DESC")
sdf_week_Umsatz.show()

+----------+---------+-----------------+
|weekofyear|dayofweek|           Umsatz|
+----------+---------+-----------------+
|        44|        6|778.2199999999999|
+----------+---------+-----------------+



In [21]:
df = sdf_week_Umsatz.toPandas()
fig = go.Figure()
fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Day of Month"),
    yaxis=dict(title_text="Count"),
    barmode="stack",
)

fig.add_trace(
        go.Bar(x=[df.dayofweek, df.weekofyear], y=df.Turnover),
    )
fig

# Korrelationsmatrix

## Ideen
- Tageszeiten
- Wochentage
- Category Class
- Anfang, Mitte, Ende des Monats
- price

## Tageszeiten - Korrelationsmatrix

In [67]:
sdf_corr_time = spark.sql("SELECT event_time, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")
sdf_corr_time = sdf_corr_time.withColumn("hour", f.hour("event_time"))
sdf_corr_time.show()           

+-------------------+--------+---------------+---------------+-------------+----+
|         event_time|turnover|bougth_quantity|viewed_quantity|cart_quantity|hour|
+-------------------+--------+---------------+---------------+-------------+----+
|2019-11-01 01:00:00|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:00|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:01|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:01|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:01|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:01|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:01|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:02|     0.0|              0|              1|            0|   1|
|2019-11-01 01:00:02|     0.0|              0|              1|            0|   1|
|2019-11-01 01:0

In [62]:
# One-hot-encoding
sdf_corr_time = sdf_corr_time.withColumn('Morning', f.when((f.col('hour')>=6) & (f.col('hour')<12), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Afternoon', f.when((f.col('hour')>=12) & (f.col('hour')<18), f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Evening', f.when(f.col('hour') > 18 , f.lit(1)).otherwise(0))
sdf_corr_time = sdf_corr_time.withColumn('Night', f.when(f.col('hour') < 6, f.lit(1)).otherwise(0))

In [63]:
sdf_corr_time = sdf_corr_time.select("Morning", "Afternoon", "Evening", "Night",  "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [64]:
sdf_corr_time.toPandas().corr().style.background_gradient(cmap='bwr')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning:

All-NaN slice encountered

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning:

All-NaN slice encountered



,Morning,Afternoon,Evening,Night,turnover,bougth_quantity,viewed_quantity,cart_quantity
Morning,nan,nan,nan,nan,nan,nan,nan,nan
Afternoon,nan,nan,nan,nan,nan,nan,nan,nan
Evening,nan,nan,nan,nan,nan,nan,nan,nan
Night,nan,nan,nan,nan,nan,nan,nan,nan
turnover,nan,nan,nan,nan,1.000000,0.909357,-0.740758,-0.005968
bougth_quantity,nan,nan,nan,nan,0.909357,1.000000,-0.814596,-0.006562
viewed_quantity,nan,nan,nan,nan,-0.740758,-0.814596,1.000000,-0.574671
cart_quantity,nan,nan,nan,nan,-0.005968,-0.006562,-0.574671,1.000000


## Wochentag - Korrelationsmatrix

In [50]:
sdf_corr_dayofweek = spark.sql("SELECT dayofweek, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")

In [51]:
# One-hot-encoding
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Sunday', f.when(f.col('dayofweek') == '1', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Monday', f.when(f.col('dayofweek') == '2', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Tuesday', f.when(f.col('dayofweek') == '3', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Wednesday', f.when(f.col('dayofweek') == '4', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Thursday', f.when(f.col('dayofweek') == '5', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Friday', f.when(f.col('dayofweek') == '6', f.lit(1)).otherwise(0))
sdf_corr_dayofweek = sdf_corr_dayofweek.withColumn('Saturday', f.when(f.col('dayofweek') == '7', f.lit(1)).otherwise(0))

sdf_corr_dayofweek = sdf_corr_dayofweek.select("Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday",  "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [53]:
sdf_corr_dayofweek.toPandas().corr().style.background_gradient(cmap='bwr')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning:

All-NaN slice encountered

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning:

All-NaN slice encountered



,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,turnover,bougth_quantity,viewed_quantity,cart_quantity
Monday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Tuesday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Wednesday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Thursday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Friday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Saturday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Sunday,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
turnover,nan,nan,nan,nan,nan,nan,nan,1.000000,0.909357,-0.740758,-0.005968
bougth_quantity,nan,nan,nan,nan,nan,nan,nan,0.909357,1.000000,-0.814596,-0.006562
viewed_quantity,nan,nan,nan,nan,nan,nan,nan,-0.740758,-0.814596,1.000000,-0.574671


## Category Class - Korrelationsmatrix

In [18]:
sdf_corr_category_class = spark.sql("SELECT category_class, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")


In [29]:
# One-hot-encoding
sdf_corr_category_class = sdf_corr_category_class.withColumn('computers', f.when(f.col('category_class') == 'computers', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('auto', f.when(f.col('category_class') == 'auto', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('apparel', f.when(f.col('category_class') == 'apparel', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('appliances', f.when(f.col('category_class') == 'appliances', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('furniture', f.when(f.col('category_class') == 'furniture', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('accessories', f.when(f.col('category_class') == 'accessories', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('electronics', f.when(f.col('category_class') == 'electronics', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('construction', f.when(f.col('category_class') == 'construction', f.lit(1)).otherwise(0))
sdf_corr_category_class = sdf_corr_category_class.withColumn('not_defined', f.when(f.col('category_class') == 'not defined', f.lit(1)).otherwise(0))

In [32]:
sdf_corr_category_class = sdf_corr_category_class.select("computers", "auto", "apparel", "appliances", "furniture", "accessories", "electronics", "construction", "not_defined", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [33]:
sdf_corr_category_class.toPandas().corr().style.background_gradient(cmap='bwr')

,computers,auto,apparel,appliances,furniture,accessories,electronics,construction,not_defined,turnover,bougth_quantity,viewed_quantity,cart_quantity
computers,1.000000,-0.062864,-0.058063,-0.090423,-0.090423,-0.023428,-0.205514,-0.067365,-0.281677,-0.030199,-0.033209,0.040768,-0.023428
auto,-0.062864,1.000000,-0.030787,-0.047946,-0.047946,-0.012423,-0.108972,-0.035720,-0.149358,-0.016013,-0.017609,0.021617,-0.012423
apparel,-0.058063,-0.030787,1.000000,-0.044284,-0.044284,-0.011474,-0.100650,-0.032992,-0.137950,-0.014790,-0.016264,0.019966,-0.011474
appliances,-0.090423,-0.047946,-0.044284,1.000000,-0.068966,-0.017869,-0.156745,-0.051379,-0.214834,-0.023033,-0.025329,0.031094,-0.017869
furniture,-0.090423,-0.047946,-0.044284,-0.068966,1.000000,-0.017869,-0.156745,-0.051379,-0.214834,0.236757,0.170968,-0.129556,-0.017869
accessories,-0.023428,-0.012423,-0.011474,-0.017869,-0.017869,1.000000,-0.040612,-0.013312,-0.055662,-0.005968,-0.006562,0.008056,-0.004630
electronics,-0.205514,-0.108972,-0.100650,-0.156745,-0.156745,-0.040612,1.000000,-0.116775,-0.488276,0.001921,0.052012,-0.108690,0.113998
construction,-0.067365,-0.035720,-0.032992,-0.051379,-0.051379,-0.013312,-0.116775,1.000000,-0.160052,-0.017159,-0.018870,0.023165,-0.013312
not_defined,-0.281677,-0.149358,-0.137950,-0.214834,-0.214834,-0.055662,-0.488276,-0.160052,1.000000,-0.071749,-0.078901,0.096859,-0.055662
turnover,-0.030199,-0.016013,-0.014790,-0.023033,0.236757,-0.005968,0.001921,-0.017159,-0.071749,1.000000,0.909357,-0.740758,-0.005968


## Monats - Korrelationsmatrix

In [68]:
sdf_corr_month = spark.sql("SELECT dayofmonth, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")

In [69]:
# One-hot-encoding
sdf_corr_month = sdf_corr_month.withColumn('Beginningofmonth', f.when(f.col('dayofmonth')<10, f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Middleofmonth', f.when((f.col('dayofmonth')>=10) & (f.col('dayofmonth')<20), f.lit(1)).otherwise(0))
sdf_corr_month = sdf_corr_month.withColumn('Endofmonth', f.when(f.col('dayofmonth') > 20 , f.lit(1)).otherwise(0))

In [70]:
sdf_corr_month = sdf_corr_month.select("Beginningofmonth", "Middleofmonth", "Endofmonth", "turnover", "bougth_quantity", "viewed_quantity", "cart_quantity" )

In [71]:
sdf_corr_month.toPandas().corr().style.background_gradient(cmap='bwr')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1264: RuntimeWarning:

All-NaN slice encountered

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\formats\style.py:1265: RuntimeWarning:

All-NaN slice encountered



,Beginningofmonth,Middleofmonth,Endofmonth,turnover,bougth_quantity,viewed_quantity,cart_quantity
Beginningofmonth,nan,nan,nan,nan,nan,nan,nan
Middleofmonth,nan,nan,nan,nan,nan,nan,nan
Endofmonth,nan,nan,nan,nan,nan,nan,nan
turnover,nan,nan,nan,1.000000,0.909357,-0.740758,-0.005968
bougth_quantity,nan,nan,nan,0.909357,1.000000,-0.814596,-0.006562
viewed_quantity,nan,nan,nan,-0.740758,-0.814596,1.000000,-0.574671
cart_quantity,nan,nan,nan,-0.005968,-0.006562,-0.574671,1.000000


## Preis-Korrelationsmatirx

In [16]:
sdf_corr_price = spark.sql("SELECT price, turnover, bougth_quantity, viewed_quantity, cart_quantity \
                            FROM Data")
sdf_corr_price.toPandas().corr().style.background_gradient(cmap='bwr')

,price,turnover,bougth_quantity,viewed_quantity,cart_quantity
price,1.000000,0.033853,0.018818,-0.033742,0.031622
turnover,0.033853,1.000000,0.909357,-0.740758,-0.005968
bougth_quantity,0.018818,0.909357,1.000000,-0.814596,-0.006562
viewed_quantity,-0.033742,-0.740758,-0.814596,1.000000,-0.574671
cart_quantity,0.031622,-0.005968,-0.006562,-0.574671,1.000000
